In [1]:
import pandas as pd
from datetime import datetime

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

import datetime as dt
import time

In [2]:
# portfolio of stocks
stocks = ['RACE', 'LVS', 'MRK', 'NIO', 'LULU', 'PYPL', 'ADBE', 'FTNT', 'CL', 'ZTS', 'LDOS', 'MNST', 'ADSK', 'MDLZ', 'ISRG', 'MRK', 'NVO', 'UNH', '4528.T', '4568.T', 'BRO', 'DEMANT.CO', 'HRMY', 'HSTM', 'TWLO', 'CRM', 'DUOL', 'GDDY', 'ACN', 'TPL', 'DOCU']

#### Download historical data from yf API

In [3]:
import yfinance as yf

start_date = "2020-01-01"
end_date = datetime.now().strftime("%Y-%m-%d")

# Download historical data from yf API
data = yf.download(stocks, start=start_date, end=end_date, group_by='ticker')

# download data to csv
data.to_csv("stocks.csv")

[*********************100%***********************]  30 of 30 completed


#### Download analysis from csv downloaded data

In [4]:
# Load the CSV with MultiIndex columns (Tickers, OHLCV)
df = pd.read_csv("stocks.csv", header=[0,1], index_col=0)

# Drop any rows that are completely NaN (e.g. 'Date' row)
df = df.dropna(how='all')

# Forward fill missing values for each column
df = df.fillna(method='ffill')

# Convert all values to float
df = df.astype(float)

# Show the result
df.head()

Ticker            LVS                                                   MDLZ  \
Price            Open       High        Low      Close     Volume       Open   
Date                                                                           
2020-01-02  65.684184  66.680544  65.599592  66.548950  5299600.0  47.902120   
2020-01-03  65.439812  65.937988  65.223618  65.533806  2822400.0  46.979939   
2020-01-06  65.026219  65.909781  64.894626  65.881577  2922500.0  47.319230   
2020-01-07  65.374021  65.984987  65.327020  65.947395  2262800.0  47.023432   
2020-01-08  65.796982  67.018929  65.693586  66.548950  3455300.0  47.006026   

Ticker                                                  ...        TWLO  \
Price            High        Low      Close     Volume  ...        Open   
Date                                                    ...               
2020-01-02  48.154417  47.075620  47.267021  7272900.0  ...  100.010002   
2020-01-03  47.475838  46.945138  47.188740  6791700.0  ...  101.099998   
2020-01-06  47.397530  46.849429  47.110432  5906400.0  ...  102.739998   
2020-01-07  47.049535  46.605833  46.910332  6716300.0  ...  107.629997   
2020-01-08  47.345325  46.901627  47.023426  5435400.0  ...  108.800003   

Ticker                                                    HRMY                 \
Price             High         Low       Close     Volume Open High Low Close   
Date                                                                            
2020-01-02  103.150002   99.660004  103.150002  2876700.0  NaN  NaN NaN   NaN   
2020-01-03  104.550003  100.760002  103.519997  2502800.0  NaN  NaN NaN   NaN   
2020-01-06  107.739998  101.769997  107.459999  3938700.0  NaN  NaN NaN   NaN   
2020-01-07  109.940002  107.370003  108.059998  2643800.0  NaN  NaN NaN   NaN   
2020-01-08  110.820000  108.199997  109.389999  3035000.0  NaN  NaN NaN   NaN   

Ticker             
Price      Volume  
Date               
2020-01-02    NaN  
2020-01-03    NaN  
2020-01-06    NaN  
2020-01-07    NaN  
2020-01-08    NaN  

[5 rows x 150 columns]

In [5]:
# # Here you can plot the charts for every ticker

# # For every ticker, plot Close
# import matplotlib.pyplot as plt

# for ticker in df.columns.levels[0]:
#     plt.figure(figsize=(12, 6))
#     plt.plot(df[ticker]['Close'], label=ticker)
#     plt.title(f"{ticker} Close Price")
#     plt.xlabel("Date")
#     plt.ylabel("Close Price")
#     plt.legend()
#     plt.show()

#### Plot of Volume Profile for All Stocks

In [6]:
import plotly.io as pio
pio.renderers.default = 'vscode'
import plotly.express as px
import plotly.graph_objects as go

for stock in stocks:
    # Use the stock data from our df
    df_stock = df[stock][['Close', 'Volume']]

    # Convert the stock to datetime if needed
    if not isinstance(df_stock.index, pd.DatetimeIndex):
        df_stock.index = pd.to_datetime(df_stock.index)

    # plot
    hist = go.Histogram(
        y=df_stock['Close'],
        x=df_stock['Volume'],
        nbinsy=150,
        orientation='h',
        name='Volume Profile',
        marker_color='#B0C4DE',
        opacity=0.5,
        xaxis='x2',
    )

    line = go.Scatter(
        x=df_stock.index,
        y=df_stock['Close'],
        mode='lines',
        name='Close',
        marker_color='blue',
        xaxis='x',
        yaxis='y',
    )

    fig = go.Figure([hist, line])

    fig.update_layout(
        xaxis=dict(
            title='Date',
            anchor='y',
            domain=[0, 1],
            showgrid=False,
            side='bottom'
        ),
        xaxis2=dict(
            title='Sum of Volume',
            overlaying='x',
            side='top',
            showgrid=False
        ),
        yaxis=dict(
            title='Close Price',
            showgrid=True
        ),
        legend=dict(
            orientation='h',
            yanchor='bottom',
            y=1.02,
            xanchor='right',
            x=1
        ),
        height=600,
        title=f'Volume Profile & {stock} Close Prices'
    )

    fig.show()